# SPARQL query

In [1]:
import pandas as pd
import numpy as np
import haversine as hs
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON, CSV

### Input parameters

In [2]:
#latitude
#longitude
#radius

#### Example setting

In [3]:
# endpoint
endpoint = 'https://query.wikidata.org/sparql'

# query variables
latitude  = '49.487777777' # Mannheim
longitude = '8.466111111'  # Mannheim

#latitude  = '49.410833333333'  # Heidelberg
#longitude = '8.7063888888889'  # Heidelberg


#obj_class = '' 
radius    = '15'
limit     = '3000'

### Query

Description of the retrieved variables in the sparql query
- ?x : Resourcee of the thing (a specific street, chruch, etc.) we search
    - ?xLabel : Label of that resource
    - ?otherLocation : Coordinates of the respective resource
- ?somebody : identifier of the person the resource is named after
    - ?somebodyLabel : Label of that person
- ?classLabel : What kind of thing are we looking at (church, street, venue...)
- ?dist : distance between current location and the location of the retrieved resource



In [4]:

# Query for the table
query = '''
        prefix bd:       <http://www.bigdata.com/rdf#>
        prefix geo:      <http://www.opengis.net/ont/geosparql#>
        prefix wd:       <http://www.wikidata.org/entity/>
        prefix wdt:      <http://www.wikidata.org/prop/direct/>
        prefix wikibase: <http://wikiba.se/ontology#>


        SELECT ?xLabel ?x ?somebodyLabel ?somebody ?otherLocation 
               (GROUP_CONCAT(?classLabel; separator=', ') AS ?classdescription)
        WHERE
        {{
          BIND('Point({longitude} {latitude})'^^geo:wktLiteral AS ?currentLocation).
          SERVICE wikibase:around {{
              ?x wdt:P625 ?otherLocation. 
              bd:serviceParam wikibase:center ?currentLocation. 
              bd:serviceParam wikibase:radius '{radius}'. 
          }}
          # x is named after
          ?x wdt:P138 ?somebody .
          
          # that somebody is a human
          ?somebody wdt:P31 wd:Q5 .

          # which Class is x (e.g. Street, church)
          ?x wdt:P31  ?class . 
          
          # the class has a german label 
          ?class rdfs:label ?classLabel .
          FILTER( lang(?classLabel) = "de" )
          
          # Retrieve Labels
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "de". }} 
          
        }} 
        GROUP BY ?xLabel ?x ?somebodyLabel ?somebody ?otherLocation
        LIMIT {limit}
        
        '''.format(longitude = longitude, latitude = latitude, radius = radius, limit = limit)
print(query)


        prefix bd:       <http://www.bigdata.com/rdf#>
        prefix geo:      <http://www.opengis.net/ont/geosparql#>
        prefix wd:       <http://www.wikidata.org/entity/>
        prefix wdt:      <http://www.wikidata.org/prop/direct/>
        prefix wikibase: <http://wikiba.se/ontology#>


        SELECT ?xLabel ?x ?somebodyLabel ?somebody ?otherLocation 
               (GROUP_CONCAT(?classLabel; separator=', ') AS ?classdescription)
        WHERE
        {
          BIND('Point(8.466111111 49.487777777)'^^geo:wktLiteral AS ?currentLocation).
          SERVICE wikibase:around {
              ?x wdt:P625 ?otherLocation. 
              bd:serviceParam wikibase:center ?currentLocation. 
              bd:serviceParam wikibase:radius '15'. 
          }
          # x is named after
          ?x wdt:P138 ?somebody .
          
          # that somebody is a human
          ?somebody wdt:P31 wd:Q5 .

          # which Class is x (e.g. Street, church)
          ?x wdt:P31  ?class . 
  

In [5]:

# Query for the map
# mapQuery = '''
#<iframe style="width: 80vw; height: 50vh; border: none;" 
#src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%20%20%20%20%20%20%20%20
#prefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0A%20%20%20%20%20%20%20%20
#prefix%20geo%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.opengis.net%2Font%2Fgeosparql%23%3E%0A%20%20%20%20%20%20%20%20
#prefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0A%20%20%20%20%20%20%20%20
#prefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0A%20%20%20%20%20%20%20%20
#prefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0A%20%20%20%20%20%20%20%20
#SELECT%20%3FxLabel%20%3Fx%20%3FsomebodyLabel%20%3Fsomebody%20%3FclassLabel%20%3FotherLocation%20%3Fdist%0A%20%20%20%20%20%20%20%20
#WHERE%0A%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20
#BIND('Point({longitude}%20{latitude})'%5E%5Egeo%3AwktLiteral%20AS%20%3FcurrentLocation).%0A%20%20%20%20%20%20%20%20%20%20
#SERVICE%20wikibase%3Aaround%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP625%20%3FotherLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20
#bd%3AserviceParam%20wikibase%3Acenter%20%3FcurrentLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20
#bd%3AserviceParam%20wikibase%3Aradius%20'{radius}'.%20%0A%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%23%20
#x%20is%20named%20after%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP138%20%3Fsomebody%20.%0A%0A%20%20%20%20%20%20%20%20%20%20%23%20
#which%20Class%20is%20x%20(e.g.%20Street%2C%20church)%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP31%20%20%3Fclass%20.%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20
#x%20is%20a%20Street%0A%20%20%20%20%20%20%20%20%20%20%23%3Fx%20wdt%3AP31%20wd%3AQ79007%20.%0A%0A%20%20%20%20%20%20%20%20%20%20%23%20
#hat%20Staat%20Deutschland%0A%20%20%20%20%20%20%20%20%20%20%23%20%3Fx%20wdt%3AP17%20%20%3FQ183%20.%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20Retrieve%20Labels%0A%20%20%20%20%20%20%20%20%20%20
#SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22de%22.%20%7D%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20
#Compute%20Distance%0A%20%20%20%20%20%20%20%20%20%20%23%20
#BIND(geof%3Adistance(%3FotherLocation%2C%20%3FcurrentLocation)%20as%20%3Fdist)%20%0A%20%20%20%20%20%20%20%20%7D%20%0A%0A%20%20%20%20%20%20%20%20
#LIMIT%20{limit}%0A%20%20%20%20%20%20%20%20%23ORDER%20BY%20%3Fdist" 
#referrerpolicy="origin" sandbox="allow-scripts allow-same-origin allow-popups"></iframe>'''.format(longitude = longitude, latitude = latitude, radius = radius, limit = limit)

mapQuery = '''
<iframe style="width: 80vw; height: 50vh; border: none;" src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%20%20%20%20%20%20%20%20prefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0A%20%20%20%20%20%20%20%20prefix%20geo%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.opengis.net%2Font%2Fgeosparql%23%3E%0A%20%20%20%20%20%20%20%20prefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0A%20%20%20%20%20%20%20%20prefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0A%20%20%20%20%20%20%20%20prefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0A%20%20%20%20%20%20%20%20SELECT%20%3FxLabel%20%3Fx%20%3FsomebodyLabel%20%3Fsomebody%20%3FotherLocation%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20(GROUP_CONCAT(%3FclassLabel%3B%20separator%3D'%2C%20')%20AS%20%3Fclassdescription)%0A%20%20%20%20%20%20%20%20WHERE%0A%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20BIND('Point({longitude}%20{latitude})'%5E%5Egeo%3AwktLiteral%20AS%20%3FcurrentLocation).%0A%20%20%20%20%20%20%20%20%20%20SERVICE%20wikibase%3Aaround%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP625%20%3FotherLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20bd%3AserviceParam%20wikibase%3Acenter%20%3FcurrentLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20bd%3AserviceParam%20wikibase%3Aradius%20'{radius}'.%20%0A%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%23%20x%20is%20named%20after%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP138%20%3Fsomebody%20.%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20that%20somebody%20is%20a%20human%0A%20%20%20%20%20%20%20%20%20%20%3Fsomebody%20wdt%3AP31%20wd%3AQ5%20.%0A%0A%20%20%20%20%20%20%20%20%20%20%23%20which%20Class%20is%20x%20(e.g.%20Street%2C%20church)%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP31%20%20%3Fclass%20.%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20the%20class%20has%20a%20german%20label%20%0A%20%20%20%20%20%20%20%20%20%20%3Fclass%20rdfs%3Alabel%20%3FclassLabel%20.%0A%20%20%20%20%20%20%20%20%20%20FILTER(%20lang(%3FclassLabel)%20%3D%20%22de%22%20)%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20Retrieve%20Labels%0A%20%20%20%20%20%20%20%20%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22de%22.%20%7D%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%7D%20%0A%20%20%20%20%20%20%20%20GROUP%20BY%20%3FxLabel%20%3Fx%20%3FsomebodyLabel%20%3Fsomebody%20%3FotherLocation%0A%20%20%20%20%20%20%20%20LIMIT%20{limit}%0A%20%20%20%20%20%20%20%20" referrerpolicy="origin" sandbox="allow-scripts allow-same-origin allow-popups"></iframe>
'''.format(longitude = longitude, latitude = latitude, radius = radius, limit = limit)

mapQuery

'\n<iframe style="width: 80vw; height: 50vh; border: none;" src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%20%20%20%20%20%20%20%20prefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0A%20%20%20%20%20%20%20%20prefix%20geo%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.opengis.net%2Font%2Fgeosparql%23%3E%0A%20%20%20%20%20%20%20%20prefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0A%20%20%20%20%20%20%20%20prefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0A%20%20%20%20%20%20%20%20prefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0A%20%20%20%20%20%20%20%20SELECT%20%3FxLabel%20%3Fx%20%3FsomebodyLabel%20%3Fsomebody%20%3FotherLocation%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20(GROUP_CONCAT(%3FclassLabel%3B%20separator%3D\'%2C%20\')%20AS%20%3Fclassdescription)%0A%20%20%20%20%20%20%20%20WHERE%0A%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20BI

In [6]:
# Initializing SPARQL Wrapper and querying
sparql = SPARQLWrapper(endpoint)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


In [7]:
# Initializing function to compute ditance
def cumputeDistance(newpoint, currentLongitude, currentLatitude):
    newLocation     = np.array([float(x) for x in newpoint[6:-1].split(' ')][::-1])
    currentLocation = np.array([float(longitude), float(latitude)][::-1])
    distance = hs.haversine(newLocation,currentLocation)
    return distance

# Results
df = pd.DataFrame(results['results']['bindings'])
for col in df.columns:
    df[col] = df[col].apply(lambda x: x['value'])

# compute ditance    
df['distance'] = df['otherLocation'].apply(lambda point: cumputeDistance(point, longitude, latitude))

# Sort according to distance
df = df.sort_values(by=['distance'])

df.head(10)



,xLabel,x,somebodyLabel,somebody,otherLocation,classdescription,distance
16,Jesuitenkirche,http://www.wikidata.org/entity/Q538329,Ignatius von Loyola,http://www.wikidata.org/entity/Q44281,Point(8.461 49.486),"Kirchengebäude, Jesuitenkirche",0.418791
52,Otto-Selz-Institut für Angewandte Psychologie,http://www.wikidata.org/entity/Q17513791,Otto Selz,http://www.wikidata.org/entity/Q74856,Point(8.468931 49.480875),"Forschungseinrichtung, Organisation",0.794126
63,Karl-Friedrich-Gymnasium Mannheim,http://www.wikidata.org/entity/Q1729755,Karl Friedrich,http://www.wikidata.org/entity/Q213671,Point(8.47603 49.4821),Gymnasium,0.954981
58,Konrad-Adenauer-Brücke,http://www.wikidata.org/entity/Q1781874,Konrad Adenauer,http://www.wikidata.org/entity/Q2492,Point(8.45638889 49.48111111),Straßenbrücke,1.021164
47,Lutherkirche,http://www.wikidata.org/entity/Q1690432,Martin Luther,http://www.wikidata.org/entity/Q9554,Point(8.46392 49.4987),"Kirchengebäude, religiöses Gebäude",1.224765
14,St. Ludwig,http://www.wikidata.org/entity/Q2320389,Ludwig IX.,http://www.wikidata.org/entity/Q346,Point(8.44853 49.4801),Kirchengebäude,1.530317
27,Melanchthonkirche,http://www.wikidata.org/entity/Q1354638,Philipp Melanchthon,http://www.wikidata.org/entity/Q76325,Point(8.4808 49.498),Kirchengebäude,1.554846
30,St.-Bernhard-Kirche,http://www.wikidata.org/entity/Q2316011,Bernhard II.,http://www.wikidata.org/entity/Q523903,Point(8.47659 49.5003),Kirchengebäude,1.584801
11,Lutherkirche,http://www.wikidata.org/entity/Q1665776,Martin Luther,http://www.wikidata.org/entity/Q9554,Point(8.44479 49.4811),Kirchengebäude,1.709849
0,Liselotte-Gymnasium Mannheim,http://www.wikidata.org/entity/Q66459147,Liselotte von der Pfalz,http://www.wikidata.org/entity/Q57221,Point(8.485 49.478417),Schule,1.716213


### Distance Test

In [8]:
## Test computing distance between two points

# Coordinates of Christuskirche  Q1087367
p1 = 'Point(8.4807 49.4846)'

# Coordinates of St-Pius-Kirche Q1405432
p2 = 'Point(8.51166 49.4798)'

loc1 = np.array([float(x) for x in p1[6:-1].split(' ')][::-1])
loc2 = np.array([float(x) for x in p2[6:-1].split(' ')][::-1])

# Calculating distance using haversine package | km per default
distance = hs.haversine(loc1,loc2)  
print('Calculated distance:        ', distance)
print('Sparql distance in Wikidata: 2.299291884545297')

# Link to control query in wikidata
# https://query.wikidata.org/#prefix%20unit%3A%20%3Chttp%3A%2F%2Fqudt.org%2Fvocab%2Funit%23%3E%0A%0ASELECT%20%3Fdist%0AWHERE%20%7B%0A%20%20%23%20Berlin%20coordinates%0A%20%20wd%3AQ1405432%20wdt%3AP625%20%3Floc1.%20%0A%20%20wd%3AQ1087367%20wdt%3AP625%20%3Floc2%20.%0A%20%20%0A%20%20BIND%28geof%3Adistance%28%3Floc1%2C%20%3Floc2%2C%20unit%3AKilometer%29%20as%20%3Fdist%29%20%0A%7D%20%0AORDER%20BY%20%3Fdist

Calculated distance:         2.2994056883957326
Sparql distance in Wikidata: 2.299291884545297


# 2.  Query: Abstract

Difficulties to retrieve <br>
#?s dbo:abstract ?abstract . <br>
#FILTER( lang(?abstract) = "en" )

In [10]:
# parameters to hand over
somebody = 'http://www.wikidata.org/entity/Q5879'  # Goethe
somebodysName = 'Martin Luther'  #linking from result set above. Necessary if no result is found
somebodysName = 'Mark Twain'

In [21]:
# Query for abstract
queryAbstract = '''
        prefix owl:  <http://www.w3.org/2002/07/owl#>
        prefix dbo:  <http://dbpedia.org/ontology>
        prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?name ?comment 
        
        WHERE {{
             ?s owl:sameAs <{somebody}> .
             ?s foaf:name ?name .
             ?s rdfs:comment ?comment.
             FILTER( lang(?comment) = "en" )
        }}
        '''.format(somebody = somebody)
print(queryAbstract)


        prefix owl:  <http://www.w3.org/2002/07/owl#>
        prefix dbo:  <http://dbpedia.org/ontology>
        prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?name ?comment # ?abstract 
        
        WHERE {
             ?s owl:sameAs <http://www.wikidata.org/entity/Q5879> .
             ?s foaf:name ?name .
             ?s rdfs:comment ?comment.
             FILTER( lang(?comment) = "en" )
        }
        


In [22]:
# Query which is searching for persons giving a string containing the requested name
queryAbstractNameSearch = '''
        prefix owl:  <http://www.w3.org/2002/07/owl#>
        prefix dbo:  <http://dbpedia.org/ontology/>
        prefix foaf: <http://xmlns.com/foaf/0.1/>
        prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT DISTINCT ?name ?comment 

        WHERE {{
            ?s rdf:type ?type .
            FILTER ( ?type = foaf:Person || ?type = dbo:Person )
            ?s foaf:name ?name .
            FILTER ( regex(?name, "{somebodysName}") ) 
            ?s rdfs:comment ?comment.
            FILTER( lang(?comment) = "en" )          
        }}
        '''.format(somebodysName = somebodysName)

In [23]:
# Query for DBPedia
endpointDBp = 'http://dbpedia.org/sparql'
sparqlDBp = SPARQLWrapper(endpointDBp)
sparqlDBp.setQuery(queryAbstract)
sparqlDBp.setReturnFormat(JSON)
resultAbstract = sparqlDBp.query().convert()

In [35]:
resultAbstract['results']['bindings'][0]['comment']['value']

'Johann Wolfgang (von) Goethe (/ˈɡɜːrtə/; German: [ˈjoːhan ˈvɔlfɡaŋ ˈɡøːtə] ; 28 August 1749 – 22 March 1832) was a German writer and statesman. His body of work includes epic and lyric poetry written in a variety of metres and styles; prose and verse dramas; memoirs; an autobiography; literary and aesthetic criticism; treatises on botany, anatomy, and colour; and four novels. In addition, numerous literary and scientific fragments, more than 10,000 letters, and nearly 3,000 drawings by him exist.'

In [26]:
# Trying to retrieve the comment.
    # In case there is none or the person was not found. Return the error message.
if resultAbstract['results']['bindings'] != []:
    # Appending names an their comments in case of several results
    comment = ''
    results = resultAbstract['results']['bindings']
    for res in results:
        comment += res['name']['value']
        comment += ':\n'
        comment += res['comment']['value']
        comment += '\n\n'
else:
    comment = 'Sorry, there is no description of {}.'.format(somebodysName)
    
print(comment[:1000])

Johann Wolfgang von Goethe:
Johann Wolfgang (von) Goethe (/ˈɡɜːrtə/; German: [ˈjoːhan ˈvɔlfɡaŋ ˈɡøːtə] ; 28 August 1749 – 22 March 1832) was a German writer and statesman. His body of work includes epic and lyric poetry written in a variety of metres and styles; prose and verse dramas; memoirs; an autobiography; literary and aesthetic criticism; treatises on botany, anatomy, and colour; and four novels. In addition, numerous literary and scientific fragments, more than 10,000 letters, and nearly 3,000 drawings by him exist.

Johann Wolfgang (von) Goethe:
Johann Wolfgang (von) Goethe (/ˈɡɜːrtə/; German: [ˈjoːhan ˈvɔlfɡaŋ ˈɡøːtə] ; 28 August 1749 – 22 March 1832) was a German writer and statesman. His body of work includes epic and lyric poetry written in a variety of metres and styles; prose and verse dramas; memoirs; an autobiography; literary and aesthetic criticism; treatises on botany, anatomy, and colour; and four novels. In addition, numerous literary and scientific fragments, mor

# 3. Query: All named 

In [54]:
queryAllResources = '''
    prefix bd:       <http://www.bigdata.com/rdf#>
    prefix wd:       <http://www.wikidata.org/entity/>
    prefix wdt:      <http://www.wikidata.org/prop/direct/>
    prefix wikibase: <http://wikiba.se/ontology#>


    SELECT ?x ?xLabel ?somebodyLabel ?Location ?countryLabel
        (GROUP_CONCAT(?classlabel; separator=', ') AS ?classdescription)

    WHERE {{
      # x is named after
      ?x wdt:P138 <{somebody}> ;

      # x has location
         wdt:P625 ?Location ;

      # which Class is x (e.g. Street, church)
         wdt:P31  ?class . 
         
      # get somebodys name (needed to display name on top of level 2)
      ?x wdt:P138 ?somebody.

      # the class has a german label 
      ?class rdfs:label ?classlabel .
      FILTER ( lang(?classlabel) = "de" )

      # located in Germany, Austria, or Switzerland
      ?x wdt:P17 ?country .
      FILTER ( ?country = wd:Q39 || ?country = wd:Q40 || ?country = wd:Q183 )

      # Retrieve Labels
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "de". }}

    }}
    GROUP BY ?x ?xLabel ?somebodyLabel ?Location ?countryLabel
    LIMIT 1000

'''.format(somebody=somebody)
print(queryAllResources)


    prefix bd:       <http://www.bigdata.com/rdf#>
    prefix wd:       <http://www.wikidata.org/entity/>
    prefix wdt:      <http://www.wikidata.org/prop/direct/>
    prefix wikibase: <http://wikiba.se/ontology#>


    SELECT ?x ?xLabel ?somebodyLabel ?Location ?countryLabel
        (GROUP_CONCAT(?classlabel; separator=', ') AS ?classdescription)

    WHERE {
      # x is named after
      ?x wdt:P138 <http://www.wikidata.org/entity/Q5879> ;

      # x has location
         wdt:P625 ?Location ;

      # which Class is x (e.g. Street, church)
         wdt:P31  ?class . 
         
      # get somebodys name (needed to display name on top of level 2)
      ?x wdt:P138 ?somebody.

      # the class has a german label 
      ?class rdfs:label ?classlabel .
      FILTER ( lang(?classlabel) = "de" )

      # located in Germany, Austria, or Switzerland
      ?x wdt:P17 ?country .
      FILTER ( ?country = wd:Q39 || ?country = wd:Q40 || ?country = wd:Q183 )

      # Retrieve Labels
      SE

In [55]:
# Query which is searching for persons giving a string containing the requested name
queryAllResourcesNameSearch = '''
    prefix bd:       <http://www.bigdata.com/rdf#>
    prefix wd:       <http://www.wikidata.org/entity/>
    prefix wdt:      <http://www.wikidata.org/prop/direct/>
    prefix wikibase: <http://wikiba.se/ontology#>


    SELECT ?x ?xLabel ?Location ?countryLabel
        (GROUP_CONCAT (DISTINCT ?classLabel; separator=', ') AS ?classdescription)

    WHERE {{
      ?s wdt:P31 wd:Q5 ;
         rdfs:label ?sLabel .
      FILTER (regex(?sLabel, "{somebodysName}") ) 
    
      # x is named after
      ?x wdt:P138 ?s ;

      # x has location
         wdt:P625 ?Location ;

      # which Class is x (e.g. Street, church)
         wdt:P31  ?class . 

      # the class has a german label 
      ?class rdfs:label ?classLabel .
      FILTER ( lang(?classLabel) = "en" )

      # located in Germany, Austria, or Switzerland
      ?x wdt:P17 ?country .
      FILTER ( ?country = wd:Q39 || ?country = wd:Q40 || ?country = wd:Q183 )

      # Retrieve Labels
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}

    }}
    GROUP BY ?x ?xLabel ?Location ?countryLabel
    LIMIT 1000

'''.format(somebodysName=somebodysName)
print(queryAllResourcesNameSearch)


    prefix bd:       <http://www.bigdata.com/rdf#>
    prefix wd:       <http://www.wikidata.org/entity/>
    prefix wdt:      <http://www.wikidata.org/prop/direct/>
    prefix wikibase: <http://wikiba.se/ontology#>


    SELECT ?x ?xLabel ?Location ?countryLabel
        (GROUP_CONCAT (DISTINCT ?classLabel; separator=', ') AS ?classdescription)

    WHERE {
      ?s wdt:P31 wd:Q5 ;
         rdfs:label ?sLabel .
      FILTER (regex(?sLabel, "Mark Twain") ) 
    
      # x is named after
      ?x wdt:P138 ?s ;

      # x has location
         wdt:P625 ?Location ;

      # which Class is x (e.g. Street, church)
         wdt:P31  ?class . 

      # the class has a german label 
      ?class rdfs:label ?classLabel .
      FILTER ( lang(?classLabel) = "en" )

      # located in Germany, Austria, or Switzerland
      ?x wdt:P17 ?country .
      FILTER ( ?country = wd:Q39 || ?country = wd:Q40 || ?country = wd:Q183 )

      # Retrieve Labels
      SERVICE wikibase:label { bd:serviceParam wikib

In [56]:
sparqlAll = SPARQLWrapper(endpoint)
sparqlAll.setQuery(queryAllResources)
sparqlAll.setReturnFormat(JSON)
resultsAll = sparqlAll.query().convert()

In [57]:
resultsAll

{'head': {'vars': ['x',
   'xLabel',
   'somebodyLabel',
   'Location',
   'countryLabel',
   'classdescription']},
 'results': {'bindings': [{'x': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q6215551'},
    'xLabel': {'xml:lang': 'de',
     'type': 'literal',
     'value': 'Johann-Wolfgang-von-Goethe-Gymnasium Chemnitz'},
    'somebodyLabel': {'xml:lang': 'de',
     'type': 'literal',
     'value': 'Johann Wolfgang von Goethe'},
    'Location': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
     'type': 'literal',
     'value': 'Point(12.942076 50.815887)'},
    'countryLabel': {'xml:lang': 'de',
     'type': 'literal',
     'value': 'Deutschland'},
    'classdescription': {'type': 'literal', 'value': 'Schule, Schulgebäude'}},
   {'x': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q19695856'},
    'xLabel': {'xml:lang': 'de',
     'type': 'literal',
     'value': 'Goethe-Schule Flensburg'},
    'somebodyLabel': {'xml:lang': 'de',
     'typ

In [58]:
# Results
dfAll = pd.DataFrame(resultsAll['results']['bindings'])
for col in dfAll.columns:
    dfAll[col] = dfAll[col].apply(lambda x: x['value'])

# compute ditance    
dfAll['distance'] = dfAll['Location'].apply(lambda point: cumputeDistance(point, longitude, latitude))

# Sort according to distance
dfAll = dfAll.sort_values(by=['distance'])

dfAll.head(10)

,x,xLabel,somebodyLabel,Location,countryLabel,classdescription,distance
38,http://www.wikidata.org/entity/Q1534105,Goethe-Gymnasium,Johann Wolfgang von Goethe,Point(8.62027778 49.69166667),Deutschland,Gymnasium,25.248516
35,http://www.wikidata.org/entity/Q22555,Goethe-Gymnasium Germersheim,Johann Wolfgang von Goethe,Point(8.37341944 49.21316111),Deutschland,Gymnasium,31.265455
32,http://www.wikidata.org/entity/Q1534232,Goethestein,Johann Wolfgang von Goethe,Point(8.16017 50.0625),Deutschland,Skulptur,67.576896
41,http://www.wikidata.org/entity/Q547717,Goetheturm,Johann Wolfgang von Goethe,Point(8.711333333 50.089),Deutschland,Aussichtsturm,69.131842
42,http://www.wikidata.org/entity/Q1507756,Goethewarte,Johann Wolfgang von Goethe,Point(8.24444 50.0931),Deutschland,Aussichtsturm,69.164213
33,http://www.wikidata.org/entity/Q50662,Goethe-Universität Frankfurt am Main,Johann Wolfgang von Goethe,Point(8.651388888 50.119444444),Deutschland,staatliche Universität,71.485674
19,http://www.wikidata.org/entity/Q1534237,Goethestraße,Johann Wolfgang von Goethe,Point(6.47598 50.8002),Deutschland,Innerortsstraße,203.478314
31,http://www.wikidata.org/entity/Q48118550,Goethehäuschen auf dem Kickelhahn,Johann Wolfgang von Goethe,Point(10.8792 50.6664),Deutschland,Holzhaus,216.379562
23,http://www.wikidata.org/entity/Q52592248,Goethestraße,Johann Wolfgang von Goethe,Point(11.44526 48.77222),Deutschland,Innerortsstraße,230.884789
34,http://www.wikidata.org/entity/Q659654,Goetheanum,Johann Wolfgang von Goethe,Point(7.620278 47.486111),Schweiz,"Schulgebäude, Mehrzweckhalle, Verwaltungsgebäu...",231.135932


In [53]:
dfAll

,x,xLabel,Location,countryLabel,classdescription,distance
39,http://www.wikidata.org/entity/Q1534105,Goethe-Gymnasium,Point(8.62027778 49.69166667),Deutschland,Gymnasium,25.248516
36,http://www.wikidata.org/entity/Q22555,Goethe-Gymnasium Germersheim,Point(8.37341944 49.21316111),Deutschland,Gymnasium,31.265455
31,http://www.wikidata.org/entity/Q1534232,Goethestein,Point(8.16017 50.0625),Deutschland,Skulptur,67.576896
34,http://www.wikidata.org/entity/Q547717,Goetheturm,Point(8.711333333 50.089),Deutschland,Aussichtsturm,69.131842
35,http://www.wikidata.org/entity/Q1507756,Goethewarte,Point(8.24444 50.0931),Deutschland,Aussichtsturm,69.164213
32,http://www.wikidata.org/entity/Q50662,Goethe-Universität Frankfurt am Main,Point(8.651388888 50.119444444),Deutschland,staatliche Universität,71.485674
18,http://www.wikidata.org/entity/Q1534237,Goethestraße,Point(6.47598 50.8002),Deutschland,Innerortsstraße,203.478314
30,http://www.wikidata.org/entity/Q48118550,Goethehäuschen auf dem Kickelhahn,Point(10.8792 50.6664),Deutschland,Holzhaus,216.379562
22,http://www.wikidata.org/entity/Q52592248,Goethestraße,Point(11.44526 48.77222),Deutschland,Innerortsstraße,230.884789
33,http://www.wikidata.org/entity/Q659654,Goetheanum,Point(7.620278 47.486111),Schweiz,"Schulgebäude, Mehrzweckhalle, Verwaltungsgebäu...",231.135932


## Corresponding Map Query

In [76]:
queryAllResourcesMap = '''

<iframe style="width: 80vw; height: 50vh; border: none;" src="https://query.wikidata.org/embed.html#%20%20%20%20%23defaultView%3AMap%0A%20%20%20%20prefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0A%20%20%20%20prefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0A%20%20%20%20prefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0A%20%20%20%20prefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0A%20%20%20%20SELECT%20%3Fx%20%3FxLabel%20%3FLocation%20%3FcountryLabel%0A%20%20%20%20%20%20%20%20(GROUP_CONCAT(%3FclassLabel%3B%20separator%3D'%2C%20')%20AS%20%3Fclassdescription)%0A%0A%20%20%20%20WHERE%20%7B%0A%20%20%20%20%20%20%23%20x%20is%20named%20after%0A%20%20%20%20%20%20%3Fx%20wdt%3AP138%20<{somebody}>%20%3B%0A%0A%20%20%20%20%20%20%23%20x%20has%20location%0A%20%20%20%20%20%20%20%20%20wdt%3AP625%20%3FLocation%20%3B%0A%0A%20%20%20%20%20%20%23%20which%20Class%20is%20x%20(e.g.%20Street%2C%20church)%0A%20%20%20%20%20%20%20%20%20wdt%3AP31%20%20%3Fclass%20.%20%0A%0A%20%20%20%20%20%20%23%20the%20class%20has%20a%20german%20label%20%0A%20%20%20%20%20%20%3Fclass%20rdfs%3Alabel%20%3FclassLabel%20.%0A%20%20%20%20%20%20FILTER%20(%20lang(%3FclassLabel)%20%3D%20%22en%22%20)%0A%0A%20%20%20%20%20%20%23%20located%20in%20Germany%2C%20Austria%2C%20or%20Switzerland%0A%20%20%20%20%20%20%3Fx%20wdt%3AP17%20%3Fcountry%20.%0A%20%20%20%20%20%20FILTER%20(%20%3Fcountry%20%3D%20wd%3AQ39%20%7C%7C%20%3Fcountry%20%3D%20wd%3AQ40%20%7C%7C%20%3Fcountry%20%3D%20wd%3AQ183%20)%0A%0A%20%20%20%20%20%20%23%20Retrieve%20Labels%0A%20%20%20%20%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%22.%20%7D%0A%0A%20%20%20%20%7D%0A%20%20%20%20GROUP%20BY%20%3Fx%20%3FxLabel%20%3FLocation%20%3FcountryLabel%0A%20%20%20%20LIMIT%201000" referrerpolicy="origin" sandbox="allow-scripts allow-same-origin allow-popups"></iframe>

'''.format(somebody=somebody)

In [17]:
queryAllResourcesNameSearchMap = '''

<iframe style="width: 80vw; height: 50vh; border: none;" src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0Aprefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0Aprefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0Aprefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0Aprefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0ASELECT%20%20%3Fx%20%3FxLabel%20%3FLocation%20%3FcountryLabel%0A(GROUP_CONCAT%20(DISTINCT%20%3FclassLabel%3B%20separator%3D'%2C%20')%20AS%20%3Fclassdescription)%0A%0AWHERE%20%7B%0A%20%20%3Fs%20wdt%3AP31%20wd%3AQ5%20%3B%0A%20%20%20%20%20rdfs%3Alabel%20%3FsLabel%20.%0A%20%20FILTER%20(regex(%3FsLabel%2C%20%22{somebodysName}%22)%20)%20%0A%0A%20%20%23%20x%20is%20named%20after%0A%20%20%3Fx%20wdt%3AP138%20%3Fs%20%3B%0A%0A%20%20%20%20%20%23%20x%20has%20location%0A%20%20%20%20%20wdt%3AP625%20%3FLocation%20%3B%0A%0A%20%20%20%20%20%23%20which%20Class%20is%20x%20(e.g.%20Street%2C%20church)%0A%20%20%20%20%20wdt%3AP31%20%20%3Fclass%20.%20%0A%0A%20%20%23%20the%20class%20has%20a%20german%20label%20%0A%20%20%3Fclass%20rdfs%3Alabel%20%3FclassLabel%20.%0A%20%20FILTER%20(%20lang(%3FclassLabel)%20%3D%20%22en%22%20)%0A%0A%20%20%23%20located%20in%20Germany%2C%20Austria%2C%20or%20Switzerland%0A%20%20%3Fx%20wdt%3AP17%20%3Fcountry%20.%0A%20%20FILTER%20(%20%3Fcountry%20%3D%20wd%3AQ39%20%7C%7C%20%3Fcountry%20%3D%20wd%3AQ40%20%7C%7C%20%3Fcountry%20%3D%20wd%3AQ183%20)%0A%0A%20%20%23%20Retrieve%20Labels%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%22.%20%7D%0A%0A%7D%0AGROUP%20BY%20%3Fx%20%3FxLabel%20%3FLocation%20%3FcountryLabel%0ALIMIT%201000" referrerpolicy="origin" sandbox="allow-scripts allow-same-origin allow-popups"></iframe>

'''.format(somebodysName=somebodysName)


###### CHANGES

- 2. Query: Abstract (2. query added)
- 2. Query: Abstract (updated method to handle results)
- 3. Query: All named (updated to en instead od de)
- 3. Query: All named (2nd query added)
- Corresponding Map Query (updated to en instead od de)
- Corresponding Map Query second query added 

In [18]:
queryAllResourcesNameSearchMap

'\n\n<iframe style="width: 80vw; height: 50vh; border: none;" src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0Aprefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0Aprefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0Aprefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0Aprefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0ASELECT%20%20%3Fx%20%3FxLabel%20%3FLocation%20%3FcountryLabel%0A(GROUP_CONCAT%20(DISTINCT%20%3FclassLabel%3B%20separator%3D\'%2C%20\')%20AS%20%3Fclassdescription)%0A%0AWHERE%20%7B%0A%20%20%3Fs%20wdt%3AP31%20wd%3AQ5%20%3B%0A%20%20%20%20%20rdfs%3Alabel%20%3FsLabel%20.%0A%20%20FILTER%20(regex(%3FsLabel%2C%20%22Konrad Adenauer%22)%20)%20%0A%0A%20%20%23%20x%20is%20named%20after%0A%20%20%3Fx%20wdt%3AP138%20%3Fs%20%3B%0A%0A%20%20%20%20%20%23%20x%20has%20location%0A%20%20%20%20%20wdt%3AP625%20%3FLocation%20%3B%0A%0A%20%20%20%20%20%23%20which%20C